In [1]:

import gym
import gym_joinemio
from gym_joinemio.envs.player import RandomPlayer
import torch
import random
from collections import defaultdict, deque
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple, deque

import numpy as np
import copy
from pprint import pprint
import math
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from joblib import Parallel, delayed
import gc

# import matplotlib
# import matplotlib.pyplot as plt
# is_ipython = 'inline' in matplotlib.get_backend()
# if is_ipython:
#     from IPython import display

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
env = gym.make('joinemio-v0')


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Replay memory

In [4]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


---

Neural net architecture:

In [5]:
class DQN(nn.Module):
    def __init__(self, h, w, outputs):    # TODO: Change params here. Different input, outputs - action space size
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

In [6]:
# Input extraction

# resize = T.Compose([T.ToPILImage(),
#                     T.Resize(40, interpolation=Image.CUBIC),
#                     T.ToTensor()])

# def get_cart_location(screen_width):
#     world_width = env.x_threshold * 2
#     scale = screen_width / world_width
#     return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART

# env.reset()
# plt.figure()
# plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(),
#            interpolation='none')
# plt.title('Example extracted screen')
# plt.show()

Training:

In [7]:
# Training
REPLAY_MEM_SIZE = 10000
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9     # EPS value will decay exponentially when training (!)
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

n_actions = env.action_space.n # 7 (Get number of actions from gym action space)


screen_height = 32 # TODO: Remove
screen_width = 32  # TODO: Remove
policy_net = DQN(screen_height, screen_width, n_actions).to(device) # TODO: Change DQN params (no height/width)
target_net = DQN(screen_height, screen_width, n_actions).to(device) 
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(REPLAY_MEM_SIZE)

steps_done = 0

def select_action(state):      # Depending on eps: action or random tensor
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row. second column on max result is 
            # index of where max element was found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

episode_durations = []

def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

NameError: name 'screen_height' is not defined

In [8]:
# Training loop | Here the training happens
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [15]:
# Loop 2
player1 = RandomPlayer()
player2 = RandomPlayer()
# iterations = 1
# for i in range(iterations): # Iterate the game
#     one_game = env.play_one_game(player1, player2, each_step_render=False)
# for e in env.recording: print(e)

num_episodes = 50
for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    # last_screen = get_screen()
    # current_screen = get_screen()
    # state = current_screen - last_screen
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        # Observe new state 
        # TODO: Use our env's state here.
        # # last_screen = current_screen
        # # current_screen = get_screen()
        # # if not done:
        # #     next_state = current_screen - last_screen
        # # else:
        # #     next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

NameError: name 'get_screen' is not defined

---

In [5]:
# def trainNNet(examples,optimizer,nnet):
#     nnet = NNET(nnet,device=device, optimizer=optimizer)
#     nnet.train(examples)
#     return nnet.nn


# class CNN(nn.Module):
#     """
#     input: 6 x 7 x 1 (reward) x ...?
#     output: 7 (best possible move?)
#     """
#     def __init__(self,action_size=7,hidden_size=128):
#         super(Connect4Net, self).__init__()

#     def forward(self, s):
#         pass

# class SampleDataset:
#     def __init__(self,samples):
#         self.samples = samples
    
#     def __len__(self):
#         return len(self.samples)
    
#     def __getitem__(self,index):
#         pass
#         return torch.tensor(object)
    
    
# class NNET:
#     """
#     Helper class to train and predict
#     """
#     def __init__(self,nnet,device,optimizer,epoch=10):
#         self.nn = nnet
#         self.epoch = epoch
#         self.device = device
#         self.optimizer = optimizer
    
#     def train(self,examples):
#         optimizer = self.optimizer
#         criterion = self.criterion
#         model = self.nn
#         device = self.device
        
#         model.to(device)
#         model.train()
#         train_loader = get_trainloader(examples)
        
#         for e in range(self.epoch):
#             total_loss = 0
#             for d in train_loader:
#                 optimizer.zero_grad()

